In [20]:
import wandb
import os
import json
import numpy as np
from environments.bounding_boxes import get_bounding_box
from utils.pareto import extreme_prune

In [21]:
api = wandb.Api(timeout=120)

In [22]:
with open('data/best_runs.json') as f:
    files = json.load(f)

for env_id, alg_dict in files.items():
    print(f'Processing {env_id}')
    _, maximals, _ = get_bounding_box(env_id)
    for alg, runs in alg_dict.items():
        print(f'Processing {alg}')
        for run_path in runs:
            run = api.run(run_path)
            seed = run.config['seed']
            pareto_points = []
            for k, v  in run.summary.items():
                if 'pareto_point' in k:
                    idx = int(k.split('_')[-1])
                    pareto_points.append((idx, np.array(v)))
            pareto_points = sorted(pareto_points, key=lambda x: x[0])
            partial_front = np.copy(maximals)
            eval_fronts = [partial_front]
            for _, point in pareto_points:
                partial_front = np.copy(np.vstack((partial_front, point)))
                eval_fronts.append(partial_front)
            
            for i, front in enumerate(eval_fronts):
                front_path = os.path.join('fronts', env_id, alg, str(seed), f'front_{i}.npy')
                os.makedirs(os.path.dirname(front_path), exist_ok=True)
                np.save(front_path, front)
            
            final_front = extreme_prune(np.copy(partial_front))
            final_front_path = os.path.join('fronts', env_id, alg, str(seed), 'final_front.npy')
            os.makedirs(os.path.dirname(final_front_path), exist_ok=True)
            np.save(final_front_path, final_front)

Processing deep-sea-treasure-concave-v0
Processing SN-MO-PPO
Processing SN-MO-DQN
Processing SN-MO-A2C
Processing minecart-v0
Processing SN-MO-PPO
Processing SN-MO-DQN
Processing SN-MO-A2C
Processing mo-reacher-v4
Processing SN-MO-PPO
Processing SN-MO-DQN
Processing SN-MO-A2C
